In [ ]:
pip install praw langdetect pandas openpyxl spacy docx
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=b1ded01961032308aceb71bc9755aa5e634937cdd27771970191dbbc4bc0bf0e
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53893 sha256=4a9e62cb2362ab1c2ea03a85d4f31317d3201ba57734586a24242cd7ad544156
  Stored in directory: /root/.cache/pip/wheels/c1/3e/c3/e81c11effd0be5658a035947c66792dd993bcff317eae0e1ed
Successfully built langdetect docx


In [ ]:
import praw
from langdetect import detect, DetectorFactory
import time

# Asegurar detección de idioma consistente
DetectorFactory.seed = 0

# Configuración de Reddit
reddit = praw.Reddit(
    client_id="u1MiQwZVrUKVb_LQzmg5yA",       # Reemplaza con tu Client ID
    client_secret="mxTNHZmHG5Yb6pPgZPpi9PWFqux0pQ",   # Reemplaza con tu Client Secret
    user_agent="script para TFM v1.0 by neupavertjm"
)

reddit.read_only = True  # Solo lectura

# Inputs del usuario
keywords = input("Introduce las palabras clave (separadas por comas): ").split(",")
keywords = [kw.strip().lower() for kw in keywords]

target_language = input("Introduce el código del idioma (ejemplo: 'es' para español): ").strip().lower()

# Subreddits de videojuegos
video_game_subreddits = ["leagueoflegends", "VALORANT", "GlobalOffensive","csgo", "cs2"]

# Términos gamer extra (para refinar búsqueda)
game_terms = ["game", "gaming", "gamer", "fps", "mmorpg", "multiplayer", "patch", "nerf", "buff", "ranked", "skin", "map","flash","feed"]

print("\n🎮 Buscando posts en subreddits de videojuegos...\n")
found_posts = 0

# Buscar en cada subreddit
for sub in video_game_subreddits:
    subreddit = reddit.subreddit(sub)
    print(f"🔍 Explorando r/{sub}...\n")

    for submission in subreddit.search(" ".join(keywords + game_terms), limit=50, sort='new', syntax='lucene'):
        try:
            text = submission.title + " " + submission.selftext
            detected_lang = detect(text)

            # ✅ Filtrar por idioma, palabra clave del input y términos gamer
            if (
                detected_lang == target_language and
                any(kw in text.lower() for kw in keywords) and
                any(term in text.lower() for term in game_terms)
            ):
                found_posts += 1
                print(f"📌 Título: {submission.title}")
                print(f"🔗 URL: {submission.url}")
                print(f"🗣️ Idioma detectado: {detected_lang}\n")

            time.sleep(1)  # Evitar rate limit

        except Exception as e:
            print(f"⚠️ Error en post: {e}")
            continue

if found_posts == 0:
    print("❌ No se encontraron posts en el idioma seleccionado y relacionados con videojuegos.")

print("\n✅ Búsqueda finalizada.")


In [ ]:
#Etiquetado de las categorías linguisticas de cada término. En este caso se optó por hacer un código, pero siempre revisado por el autor de la investigación.
import spacy
import json
import csv
from collections import Counter

# Cargar el modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

# Leer el JSON con "Término en español", "Término en inglés" y "Definición"
with open("terminos.json", "r", encoding="utf-8") as f:
    datos = json.load(f)

# Diccionario para traducir etiquetas POS al español
etiquetas = {
    "NOUN": "sustantivo",
    "VERB": "verbo",
    "ADJ": "adjetivo",
    "ADV": "adverbio"
}

# Diccionario de términos en inglés y su categoría asociada
terminos_en_ingles = {
    "gank": "verbo",
    "feed": "verbo",
    "rush": "verbo",
    # Agregar más términos en inglés si es necesario
}

# Procesar términos
resultados = []

for entrada in datos:
    termino_es = entrada.get("Término en español", "").strip()
    termino_en = entrada.get("Término en inglés", "").strip()
    definicion = entrada.get("Definición", "").strip()

    categoria_principal = None

    # Comprobar si el término en inglés tiene una categoría predefinida
    if termino_en in terminos_en_ingles:
        categoria_principal = terminos_en_ingles[termino_en]
    else:
        # Procesar el término en español usando spaCy
        doc = nlp(definicion)
        conteo_categorias = Counter()

        for token in doc:
            if token.pos_ in etiquetas:
                conteo_categorias[etiquetas[token.pos_]] += 1

        if conteo_categorias:
            categoria_principal = conteo_categorias.most_common(1)[0][0]
        else:
            categoria_principal = "otra"

    resultados.append({
        "Término en español": termino_es,
        "Término en inglés": termino_en,
        "Definición": definicion,
        "Categoría lingüística": categoria_principal
    })

# Guardar como CSV
with open("terminos_con_categorias.csv", "w", encoding="utf-8", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["Término en español", "Término en inglés", "Definición", "Categoría lingüística"])
    writer.writeheader()
    writer.writerows(resultados)

# Mostrar algunos resultados por consola
for r in resultados[:10]:
    print(f"{r['Término en español']} / {r['Término en inglés']}: {r['Categoría lingüística']}")


OTP / : verbo
Mejora / Bufo / : sustantivo
Golpeo a Distancia (Desgastar?) / : sustantivo
Potenciador (?) / Boost / : sustantivo
Draft / : sustantivo
Botlane (bot) /linea inferior / : sustantivo
Botlane (bot) /linea inferior / : sustantivo
Ace / : sustantivo
Grupo / Formación / : sustantivo
Stunnear / Aturdir / : sustantivo


In [ ]:
pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.8 MB/s eta 0:00:00
